In [2]:
import numpy as np
import pandas as pd
import random

In [3]:
file_path = '../data/nfl_scores.csv'

data = pd.read_csv(file_path)
nfl_df = pd.DataFrame(data)


In [4]:
def calc_expected_res(team_rating, opp_rating):
  return 1 / (1 + 10 ** ((opp_rating - team_rating) / 400))

In [5]:
# Given home and away team update elo
def update_rating(team_rating, opp_rating, res, k=20):
  expected = calc_expected_res(team_rating, opp_rating)
  return team_rating + k * (res - expected)

In [6]:
def calc_elo_rankings(games):
  temp = pd.DataFrame({'teams': games['team_home'].unique(),
                       'elo': 1500})

  for i, game in games.iterrows():
    home_team = game['team_home']
    away_team = game['team_away']

    score_home = game['score_home']
    score_away = game['score_away']

    home_elo = temp.loc[temp['teams'] == home_team, 'elo'].iloc[0]
    away_elo = temp.loc[temp['teams'] == away_team, 'elo'].iloc[0]

    res = 1 if score_home > score_away else 0
    new_home_elo = update_rating(home_elo, away_elo, res)
    new_away_elo = update_rating(away_elo, home_elo, 1 - res)

    temp.loc[temp['teams'] == home_team, 'elo'] = new_home_elo
    temp.loc[temp['teams'] == away_team, 'elo'] = new_away_elo

  return temp
  

In [7]:
team_names = {'Washington Redskins': 'Washington Commanders', 
              'Washington Football Team': 'Washington Commanders', 
              'Oakland Raiders': 'Las Vegas Raiders',
              'Houston Oilers': 'Tennessee Titans',
              'Tennessee Oilers': 'Tennessee Titans',
              'St. Louis Rams': 'Los Angeles Rams',
              'St. Louis Cardinals': 'Arizona Cardinals',
              'Phoenix Cardinals': 'Arizona Cardinals',
              'San Diego Chargers': 'Los Angeles Chargers',
              'Baltimore Colts': 'Baltimore Ravens',
              'Boston Patriots': 'New England Patriots',
              'Los Angeles Raiders': 'Las Vegas Raiders'}

nfl_df['team_home'] = nfl_df['team_home'].replace(team_names)
nfl_df['team_away'] = nfl_df['team_away'].replace(team_names)

In [8]:
nfl_df["result_home"] = nfl_df["score_home"] - nfl_df["score_away"]
nfl_2020_2022 = nfl_df[(nfl_df['schedule_season'] >= 2020) & (nfl_df['schedule_season'] < 2023)]
display(nfl_2020_2022['team_home'].unique())
print(len(nfl_2020_2022['team_home'].unique()))

array(['Kansas City Chiefs', 'Atlanta Falcons', 'Baltimore Ravens',
       'Buffalo Bills', 'Carolina Panthers', 'Cincinnati Bengals',
       'Detroit Lions', 'Jacksonville Jaguars', 'Los Angeles Rams',
       'Minnesota Vikings', 'New England Patriots', 'New Orleans Saints',
       'San Francisco 49ers', 'Washington Commanders', 'Denver Broncos',
       'New York Giants', 'Cleveland Browns', 'Arizona Cardinals',
       'Chicago Bears', 'Dallas Cowboys', 'Green Bay Packers',
       'Houston Texans', 'Indianapolis Colts', 'Los Angeles Chargers',
       'Miami Dolphins', 'New York Jets', 'Philadelphia Eagles',
       'Pittsburgh Steelers', 'Seattle Seahawks', 'Tampa Bay Buccaneers',
       'Tennessee Titans', 'Las Vegas Raiders'], dtype=object)

32


In [129]:
def season_simulation(games, elo_ratings, num_sim=1000):
    sim_outcome = pd.DataFrame({'Teams': elo_ratings['teams'].unique(),
                                'Wins': 0,
                                'Loss': 0})

    sim_outcome2 = pd.DataFrame({'Teams': elo_ratings['teams'].unique(),
                                'Wins': 0,
                                'Loss': 0})

    start_week = 14
    for week in range(1, start_week):
        game_week = games[games['week'] == week]
        for i, game in game_week.iterrows():
            home_team = game['home_team']
            away_team = game['away_team']
            result_bool = game['result_home']
            res = 1 if result_bool else 0
            sim_outcome.loc[sim_outcome['Teams'] == home_team, 'Wins'] += res
            sim_outcome.loc[sim_outcome['Teams'] == home_team, 'Loss'] += (1 - res)
            sim_outcome.loc[sim_outcome['Teams'] == away_team, 'Wins'] += (1 - res)
            sim_outcome.loc[sim_outcome['Teams'] == away_team, 'Loss'] += res

            home_elo = elo_ratings.loc[elo_ratings['teams'] == home_team, 'elo'].iloc[0]
            away_elo = elo_ratings.loc[elo_ratings['teams'] == away_team, 'elo'].iloc[0]

            new_home_elo = update_rating(home_elo, away_elo, res, k=40)
            new_away_elo = update_rating(away_elo, home_elo, 1 - res, k=40)

            elo_ratings.loc[elo_ratings['teams'] == home_team, 'elo'] = new_home_elo
            elo_ratings.loc[elo_ratings['teams'] == away_team, 'elo'] = new_away_elo

    # display(sim_outcome.sort_values(by='Wins', ascending=False))
    # display(elo_ratings.sort_values(by='elo', ascending=False))

    for sim in range(num_sim):
        if sim % 10 == 0:
            print("Simulation:", sim)
        elo = elo_ratings.copy()
        for week_index in range(start_week, 19):
            curr = games[games['week'] == week_index]
            for i, game in curr.iterrows():
                home_team = game['home_team']
                away_team = game['away_team']

                home_elo = elo.loc[elo['teams'] == home_team, 'elo'].iloc[0]
                away_elo = elo.loc[elo['teams'] == away_team, 'elo'].iloc[0]

                # Prediction here
                expected = calc_expected_res(home_elo, away_elo)
                res = 1 if random.random() < expected else 0

                sim_outcome2.loc[sim_outcome2['Teams'] == home_team, 'Wins'] += res
                sim_outcome2.loc[sim_outcome2['Teams'] == home_team, 'Loss'] += (1 - res)
                sim_outcome2.loc[sim_outcome2['Teams'] == away_team, 'Wins'] += (1 - res)
                sim_outcome2.loc[sim_outcome2['Teams'] == away_team, 'Loss'] += res
    
                # Update model
                new_home_elo = update_rating(home_elo, away_elo, res)
                new_away_elo = update_rating(away_elo, home_elo, 1 - res)

                elo.loc[elo['teams'] == home_team, 'elo'] = new_home_elo
                elo.loc[elo['teams'] == away_team, 'elo'] = new_away_elo

    win_ratio = (sim_outcome2['Wins'] / num_sim)
    loss_ratio = (sim_outcome2['Loss'] / num_sim)
    sim_outcome['Wins'] += win_ratio
    sim_outcome['Loss'] += loss_ratio
    return sim_outcome

In [130]:
elo_data = calc_elo_rankings(nfl_2020_2022)
# display(elo_data.sort_values(by='elo', ascending=False))
nfl_2023 = pd.read_csv('../data/nfl_scrapped_2023.csv')
pred = season_simulation(nfl_2023, elo_data)

/var/folders/8s/x__k3qxs3lbgfsdfz_rp19qw0000gn/T/ipykernel_13881/175792531.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1519.4249887221547' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp.loc[temp['teams'] == home_team, 'elo'] = new_home_elo


Simulation: 0
Simulation: 10
Simulation: 20
Simulation: 30
Simulation: 40
Simulation: 50
Simulation: 60
Simulation: 70
Simulation: 80
Simulation: 90
Simulation: 100
Simulation: 110
Simulation: 120
Simulation: 130
Simulation: 140
Simulation: 150
Simulation: 160
Simulation: 170
Simulation: 180
Simulation: 190
Simulation: 200
Simulation: 210
Simulation: 220
Simulation: 230
Simulation: 240
Simulation: 250
Simulation: 260
Simulation: 270
Simulation: 280
Simulation: 290
Simulation: 300
Simulation: 310
Simulation: 320
Simulation: 330
Simulation: 340
Simulation: 350
Simulation: 360
Simulation: 370
Simulation: 380
Simulation: 390
Simulation: 400
Simulation: 410
Simulation: 420
Simulation: 430
Simulation: 440
Simulation: 450
Simulation: 460
Simulation: 470
Simulation: 480
Simulation: 490
Simulation: 500
Simulation: 510
Simulation: 520
Simulation: 530
Simulation: 540
Simulation: 550
Simulation: 560
Simulation: 570
Simulation: 580
Simulation: 590
Simulation: 600
Simulation: 610
Simulation: 620
Sim

In [133]:
pred.sort_values(by="Wins", ascending=False).tail(16)

,Teams,Wins,Loss
14,Denver Broncos,8.527,8.473
9,Minnesota Vikings,8.417,8.583
28,Seattle Seahawks,8.288,8.712
29,Tampa Bay Buccaneers,7.649,9.351
11,New Orleans Saints,7.637,9.363
5,Cincinnati Bengals,7.376,9.624
23,Los Angeles Chargers,7.235,9.765
31,Las Vegas Raiders,7.080,9.920
25,New York Jets,5.913,11.087
30,Tennessee Titans,5.876,11.124


In [116]:
afc_team_data = {
    "Division": {
        "East": ["Buffalo Bills", "Miami Dolphins", "New England Patriots", "New York Jets"],
        "North": ["Baltimore Ravens", "Cincinnati Bengals", "Cleveland Browns", "Pittsburgh Steelers"],
        "South": ["Houston Texans", "Indianapolis Colts", "Jacksonville Jaguars", "Tennessee Titans"],
        "West": ["Denver Broncos", "Kansas City Chiefs", "Las Vegas Raiders", "Los Angeles Chargers"]
    }
}

afc_df = pd.DataFrame()

for division, teams in afc_team_data["Division"].items():
    for team in teams:
        afc_df = pd.concat([afc_df, pd.DataFrame({"Teams": [team], "Conference": ["AFC"], "Division": [division]})])

afc_df.reset_index(drop=True, inplace=True)

print(afc_df)


                   Teams Conference Division
0          Buffalo Bills        AFC     East
1         Miami Dolphins        AFC     East
2   New England Patriots        AFC     East
3          New York Jets        AFC     East
4       Baltimore Ravens        AFC    North
5     Cincinnati Bengals        AFC    North
6       Cleveland Browns        AFC    North
7    Pittsburgh Steelers        AFC    North
8         Houston Texans        AFC    South
9     Indianapolis Colts        AFC    South
10  Jacksonville Jaguars        AFC    South
11      Tennessee Titans        AFC    South
12        Denver Broncos        AFC     West
13    Kansas City Chiefs        AFC     West
14     Las Vegas Raiders        AFC     West
15  Los Angeles Chargers        AFC     West


In [117]:
nfc_team_data = {
    "Division": {
        "East": ["Dallas Cowboys", "New York Giants", "Philadelphia Eagles", "Washington Commanders"],
        "North": ["Chicago Bears", "Detroit Lions", "Green Bay Packers", "Minnesota Vikings"],
        "South": ["Atlanta Falcons", "Carolina Panthers", "New Orleans Saints", "Tampa Bay Buccaneers"],
        "West": ["Arizona Cardinals", "Los Angeles Rams", "San Francisco 49ers", "Seattle Seahawks"]
    }
}

nfc_df = pd.DataFrame()

for division, teams in nfc_team_data["Division"].items():
    for team in teams:
        nfc_df = pd.concat([nfc_df, pd.DataFrame({"Teams": [team], "Conference": ["NFC"], "Division": [division]})])

nfc_df.reset_index(drop=True, inplace=True)
display(nfc_df)

,Teams,Conference,Division
0,Dallas Cowboys,NFC,East
1,New York Giants,NFC,East
2,Philadelphia Eagles,NFC,East
3,Washington Commanders,NFC,East
4,Chicago Bears,NFC,North
5,Detroit Lions,NFC,North
6,Green Bay Packers,NFC,North
7,Minnesota Vikings,NFC,North
8,Atlanta Falcons,NFC,South
9,Carolina Panthers,NFC,South


In [118]:
combined_teams_df = pd.concat([afc_df, nfc_df])
afc_df = pd.merge(afc_df, pred, on='Teams')
nfc_df = pd.merge(nfc_df, pred, on='Teams')
nfc_df

,Teams,Conference,Division,Wins,Loss
0,Dallas Cowboys,NFC,East,11.98,5.02
1,New York Giants,NFC,East,5.61,11.39
2,Philadelphia Eagles,NFC,East,13.62,3.38
3,Washington Commanders,NFC,East,5.35,11.65
4,Chicago Bears,NFC,North,5.69,11.31
5,Detroit Lions,NFC,North,11.79,5.21
6,Green Bay Packers,NFC,North,9.40,7.60
7,Minnesota Vikings,NFC,North,8.53,8.47
8,Atlanta Falcons,NFC,South,8.45,8.55
9,Carolina Panthers,NFC,South,2.25,14.75


In [55]:
def get_playoff_df(conf_record):
  # Select best record in each division
  division_champs = conf_record.sort_values(by=['Wins', 'Loss'], ascending=[False, True]) \
                               .groupby('Division').head(1)
  division_champs["Seed"] = [1,2,3,4]
  # Select 3 best remaining records as wildcard
  wild_cards = conf_record.sort_values(by=['Wins', 'Loss'], ascending=[False, True]) \
                          .groupby('Division').tail(3).head(3)
  wild_cards["Seed"] = [5,6,7]
  return pd.concat([division_champs, wild_cards]).reset_index(drop=True)

In [119]:
afc_playoff = get_playoff_df(afc_df)
nfc_playoff = get_playoff_df(nfc_df)

In [120]:
afc_playoff

,Teams,Conference,Division,Wins,Loss,Seed
0,Jacksonville Jaguars,AFC,South,12.11,4.89,1
1,Miami Dolphins,AFC,East,11.92,5.08,2
2,Baltimore Ravens,AFC,North,11.68,5.32,3
3,Kansas City Chiefs,AFC,West,11.57,5.43,4
4,Cleveland Browns,AFC,North,9.78,7.22,5
5,Pittsburgh Steelers,AFC,North,9.70,7.30,6
6,Houston Texans,AFC,South,9.62,7.38,7


In [121]:
nfc_playoff

,Teams,Conference,Division,Wins,Loss,Seed
0,Philadelphia Eagles,NFC,East,13.62,3.38,1
1,San Francisco 49ers,NFC,West,12.78,4.22,2
2,Detroit Lions,NFC,North,11.79,5.21,3
3,Atlanta Falcons,NFC,South,8.45,8.55,4
4,Dallas Cowboys,NFC,East,11.98,5.02,5
5,Green Bay Packers,NFC,North,9.40,7.60,6
6,Los Angeles Rams,NFC,West,8.63,8.37,7


In [122]:
nfl_playoff = pd.concat([nfc_playoff, afc_playoff])
nfl_playoff

,Teams,Conference,Division,Wins,Loss,Seed
0,Philadelphia Eagles,NFC,East,13.62,3.38,1
1,San Francisco 49ers,NFC,West,12.78,4.22,2
2,Detroit Lions,NFC,North,11.79,5.21,3
3,Atlanta Falcons,NFC,South,8.45,8.55,4
4,Dallas Cowboys,NFC,East,11.98,5.02,5
5,Green Bay Packers,NFC,North,9.40,7.60,6
6,Los Angeles Rams,NFC,West,8.63,8.37,7
0,Jacksonville Jaguars,AFC,South,12.11,4.89,1
1,Miami Dolphins,AFC,East,11.92,5.08,2
2,Baltimore Ravens,AFC,North,11.68,5.32,3


In [60]:
def generate_matchups(playoff_teams, pairings):
  matchups = pd.DataFrame(pairings)
  matchups = matchups.merge(playoff_teams, left_on="team_home_seed", right_on="Seed", suffixes=('_home', '_away')) \
                     .merge(playoff_teams, left_on="team_away_seed", right_on="Seed", suffixes=('_home', '_away'))

  matchups = matchups[['team_home_seed', 'team_away_seed', 'Teams_home', 'Teams_away']]
  matchups.columns = ['team_home_seed', 'team_away_seed', 'team_home', 'team_away']
  return matchups

In [61]:
def generate_round_one(playoff_teams):
  pairings = {
      "team_home_seed": [2, 3, 4],
      "team_away_seed": [7, 6, 5]
  }
  return generate_matchups(playoff_teams, pairings)

In [62]:
def simulate_postseason_round(curr_round, teams, elo):
  sim_outcome = pd.DataFrame()

  for _, game in curr_round.iterrows():
      home_team = game['team_home']
      away_team = game['team_away']

      home_elo = elo.loc[elo['teams'] == home_team, 'elo'].iloc[0]
      away_elo = elo.loc[elo['teams'] == away_team, 'elo'].iloc[0]

      # Prediction here
      expected = calc_expected_res(home_elo, away_elo)
      res = 1 if random.random() < expected else 0
      if res:
        row = teams.loc[teams['Teams'] == home_team]
        sim_outcome = pd.concat([sim_outcome, row])
      else:
        row = teams.loc[teams['Teams'] == away_team]
        sim_outcome = pd.concat([sim_outcome, row])

      new_home_elo = update_rating(home_elo, away_elo, res)
      new_away_elo = update_rating(away_elo, home_elo, 1 - res)
    
      # Update model
      elo.loc[elo['teams'] == home_team, 'elo'] = new_home_elo
      elo.loc[elo['teams'] == away_team, 'elo'] = new_away_elo

  return pd.concat([sim_outcome]).reset_index(drop=True)

In [63]:
def round1_simulation(wildcard, teams, elo):
  sim_outcome = pd.DataFrame()

  for _, game in wildcard.iterrows():

      home_team = game['team_home']
      away_team = game['team_away']

      home_elo = elo.loc[elo['teams'] == home_team, 'elo'].iloc[0]
      away_elo = elo.loc[elo['teams'] == away_team, 'elo'].iloc[0]

      expected = calc_expected_res(home_elo, away_elo)
      res = 1 if random.random() < expected else 0

      if res:
        row = teams.loc[teams['Teams'] == home_team]
        sim_outcome = pd.concat([sim_outcome, row])
      else:
        row = teams.loc[teams['Teams'] == away_team]
        sim_outcome = pd.concat([sim_outcome, row])

      new_home_elo = update_rating(home_elo, away_elo, res)
      new_away_elo = update_rating(away_elo, home_elo, 1 - res)
    
      # Update model
      elo.loc[elo['teams'] == home_team, 'elo'] = new_home_elo
      elo.loc[elo['teams'] == away_team, 'elo'] = new_away_elo

  return pd.concat([sim_outcome, teams.head(1)]).reset_index(drop=True)

In [64]:
def generate_round_two(round1_results):
    order = round1_results["Seed"].sort_values().reset_index(drop=True)
    pairings = {
        "team_home_seed": [order[0], order[1]],
        "team_away_seed": [order[3], order[2]]
    }
    return generate_matchups(round1_results, pairings)

In [65]:
def generate_division_champ(results):
    order = results["Seed"].sort_values().reset_index(drop=True)
    pairings = {
        "team_home_seed": [order[0]],
        "team_away_seed": [order[1]]
    }
    return generate_matchups(results, pairings)

In [66]:
def generate_superbowl(results):
    matchup = pd.DataFrame({
        "team_home_seed": [str(results["Seed"].iloc[0])],
        "team_away_seed": [str(results["Seed"].iloc[1])],
        "team_home": [results["Teams"].iloc[0]],
        "team_away": [results["Teams"].iloc[1]]
    })
    return matchup

In [67]:
def simulate_playoffs(conf, elo_data):
  round1 = generate_round_one(conf)
  round2 = round1_simulation(round1, conf, elo_data)
  divisional = generate_round_two(round2)
  round3 = simulate_postseason_round(divisional, conf, elo_data)
  finals = generate_division_champ(round3)
  return simulate_postseason_round(finals, conf, elo_data)

def predict_super_bowl_winner(conf1, conf2, elo_data):
  superbowl = pd.concat([simulate_playoffs(nfc_playoff, elo_data), simulate_playoffs(afc_playoff, elo_data)])
  superbowl_matchup = generate_superbowl(superbowl)
  return simulate_postseason_round(superbowl_matchup, pd.concat([conf1, conf2]), elo_data) 

In [123]:
round1 = generate_round_one(nfc_playoff)
round1_results = round1_simulation(round1, nfc_playoff, elo_data)
predict_super_bowl_winner(nfc_playoff, afc_playoff, elo_data)

,Teams,Conference,Division,Wins,Loss,Seed
0,Miami Dolphins,AFC,East,11.92,5.08,2


In [124]:
def playoff_simulation(conf1, conf2, elo_rankings, teams, num_sim=5000):
  sim_outcome = pd.DataFrame({'Teams': teams['Teams'].unique(),
                              'Round 1': 0,
                              'Round 2': 0,
                              'Conference': 0,
                              'Superbowl': 0})
  
  for i in range(num_sim):
    elo_data = elo_rankings.copy()
    if i % 100 == 0:
      print("Simulation:", i)
    # Simulate Conf1 Playoff bracket 
    round1_pairings = generate_round_one(conf1)
    round1_results = round1_simulation(round1_pairings, conf1, elo_data)
    for _, result in round1_results.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Round 1'] += 1

    round2_pairings = generate_round_two(round1_results)
    round2_results = simulate_postseason_round(round2_pairings, conf1, elo_data)

    for _, result in round2_results.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Round 2'] += 1

    conf_pairing = generate_division_champ(round2_results)
    conf_results = simulate_postseason_round(conf_pairing, conf1, elo_data)

    for _, result in conf_results.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Conference'] += 1
    
    # Simulate Conf2 Playoff bracket 
    round1_pairings_conf2 = generate_round_one(conf2)
    round1_results_conf2 = round1_simulation(round1_pairings_conf2, conf2, elo_data)
    for _, result in round1_results_conf2.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Round 1'] += 1

    round2_pairings_conf2 = generate_round_two(round1_results_conf2)
    round2_results_conf2 = simulate_postseason_round(round2_pairings_conf2, conf2, elo_data)

    for _, result in round2_results_conf2.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Round 2'] += 1

    conf_pairing_conf2 = generate_division_champ(round2_results_conf2)
    conf_results_conf2 = simulate_postseason_round(conf_pairing_conf2, conf2, elo_data)

    for _, result in conf_results_conf2.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Conference'] += 1

    # Simulate superbowl
    superbowl_teams = pd.concat([conf_results, conf_results_conf2])
    superbowl_pairing = generate_superbowl(superbowl_teams)
    superbowl_result = simulate_postseason_round(superbowl_pairing, pd.concat([conf1, conf2]), elo_data) 

    for _, result in superbowl_result.iterrows():
      sim_outcome.loc[result['Teams'] == sim_outcome['Teams'], 'Superbowl'] += 1
  
  sim_outcome['Round 1'] /= num_sim
  sim_outcome['Round 2'] /= num_sim
  sim_outcome['Conference'] /= num_sim
  sim_outcome['Superbowl'] /= num_sim
  return sim_outcome

# display(elo_data.sort_values(by='elo', ascending=False))
playoff_sim = playoff_simulation(nfc_playoff, afc_playoff, elo_data, nfl_playoff)
display(playoff_sim.sort_values(by='Superbowl', ascending=False))

Simulation: 0
Simulation: 100
Simulation: 200
Simulation: 300
Simulation: 400
Simulation: 500
Simulation: 600
Simulation: 700
Simulation: 800
Simulation: 900
Simulation: 1000
Simulation: 1100
Simulation: 1200
Simulation: 1300
Simulation: 1400
Simulation: 1500
Simulation: 1600
Simulation: 1700
Simulation: 1800
Simulation: 1900
Simulation: 2000
Simulation: 2100
Simulation: 2200
Simulation: 2300
Simulation: 2400
Simulation: 2500
Simulation: 2600
Simulation: 2700
Simulation: 2800
Simulation: 2900
Simulation: 3000
Simulation: 3100
Simulation: 3200
Simulation: 3300
Simulation: 3400
Simulation: 3500
Simulation: 3600
Simulation: 3700
Simulation: 3800
Simulation: 3900
Simulation: 4000
Simulation: 4100
Simulation: 4200
Simulation: 4300
Simulation: 4400
Simulation: 4500
Simulation: 4600
Simulation: 4700
Simulation: 4800
Simulation: 4900


,Teams,Round 1,Round 2,Conference,Superbowl
0,Philadelphia Eagles,1.0000,0.6368,0.3522,0.2000
1,San Francisco 49ers,0.6632,0.4234,0.2330,0.1412
10,Kansas City Chiefs,0.6786,0.4146,0.2554,0.1402
8,Miami Dolphins,0.7384,0.3932,0.2310,0.1184
4,Dallas Cowboys,0.7110,0.3324,0.1622,0.0798
7,Jacksonville Jaguars,1.0000,0.4950,0.1950,0.0794
9,Baltimore Ravens,0.5974,0.2762,0.1532,0.0714
5,Green Bay Packers,0.5450,0.2386,0.1012,0.0468
2,Detroit Lions,0.4550,0.1786,0.0740,0.0338
12,Pittsburgh Steelers,0.4026,0.1826,0.0770,0.0278


In [128]:
for i, row in playoff_sim.iterrows():
  print(row['Teams'])

Philadelphia Eagles
San Francisco 49ers
Detroit Lions
Atlanta Falcons
Dallas Cowboys
Green Bay Packers
Los Angeles Rams
Jacksonville Jaguars
Miami Dolphins
Baltimore Ravens
Kansas City Chiefs
Cleveland Browns
Pittsburgh Steelers
Houston Texans


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [40]:
nfl_data = nfl_2018.dropna(axis=1)
nfl_data = nfl_data.drop('schedule_date', axis=1)
nfl_data

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,result_home
12411,2019,1,False,Chicago Bears,3.0,10.0,Green Bay Packers,CHI,-3.0,46.5,Soldier Field,False,-7.0
12412,2019,1,False,Arizona Cardinals,27.0,27.0,Detroit Lions,DET,-3.0,45.5,University of Phoenix Stadium,False,0.0
12413,2019,1,False,Carolina Panthers,27.0,30.0,Los Angeles Rams,LAR,-2.0,49.5,Bank of America Stadium,False,-3.0
12414,2019,1,False,Cleveland Browns,13.0,43.0,Tennessee Titans,CLE,-5.5,44,FirstEnergy Stadium,False,-30.0
12415,2019,1,False,Dallas Cowboys,35.0,17.0,New York Giants,DAL,-7.0,44,AT&T Stadium,False,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227,2021,Division,True,Kansas City Chiefs,42.0,36.0,Buffalo Bills,KC,-2.5,54,Arrowhead Stadium,False,6.0
13228,2021,Division,True,Tampa Bay Buccaneers,27.0,30.0,Los Angeles Rams,TB,-3.0,48,Raymond James Stadium,False,-3.0
13229,2021,Conference,True,Kansas City Chiefs,24.0,27.0,Cincinnati Bengals,KC,-7.0,54.5,Arrowhead Stadium,False,-3.0
13230,2021,Conference,True,Los Angeles Rams,20.0,17.0,San Francisco 49ers,LAR,-3.5,46,SoFi Stadium,False,3.0


In [41]:
X = nfl_data.drop('result_home', axis=1)
y = nfl_data['result_home']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
nfl_labels = nfl_data["result_home"].tolist()
nfl_features = nfl_data._get_numeric_data().columns.values.tolist()[1:-1]
nfl_feature_data = nfl_data[nfl_features]
X_train, X_test, y_train, y_test = train_test_split(nfl_feature_data, nfl_labels, test_size=0.3)
nfl_features

['schedule_playoff',
 'score_home',
 'score_away',
 'spread_favorite',
 'stadium_neutral']

In [43]:
params = {
    "n_estimators": 500,
    "learning_rate": 0.01,
    "loss": "log_loss",
}

In [44]:
gb_model = GradientBoostingClassifier(**params)

# Train the Gradient Boosting model
gb_model.fit(X_train, y_train)

# Make predictions on the test set
gb_predictions = gb_model.predict(X_test)

# Evaluate the Gradient Boosting model accuracy
gb_accuracy = accuracy_score(y_test, gb_predictions)
print(f'Gradient Boosting Model Accuracy: {gb_accuracy}')

Gradient Boosting Model Accuracy: 0.2793522267206478


In [45]:
rf_model = RandomForestClassifier(n_estimators=1000)

rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f'Random Forest Model Accuracy: {rf_accuracy}')

Random Forest Model Accuracy: 0.2591093117408907
